# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import datetime

import numpy as np
import pandas as pd

from src.data import train_test_split, MRISequence
from src.model import create_model, compile_model, load_checkpoint
from src.model.evaluation import show_metrics

In [97]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['image.cmap'] = 'viridis'

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 15})

In [212]:
from src.evaluation.segmentation_masks import SegmentationMasksSaver

ROOT_DIR = '../../tmp'

saver = SegmentationMasksSaver(ROOT_DIR)

In [200]:
# saver.df.head()

In [192]:
# list(saver.df.columns)

In [213]:
df = saver.df

def ins_del_fn(ins_col, del_col):
    def fn(row):
        value = (row[ins_col] + (1 - row[del_col])) / 2
        return value
    return fn

for cls in ['AD+CN', 'AD', 'CN']:
# for cls in ['AD+CN']:
    df[f'ins_del__auc_median__{cls}'] = df.apply(ins_del_fn(f'insertion__auc_median__{cls}', f'deletion__auc_median__{cls}'), axis=1)

In [214]:
columns = ['notebook_key']

# for cls in ['AD+CN']: 
for cls in ['AD+CN', 'AD', 'CN']: 
    for m in [
        'ins_del__auc_median__',
#         'arr_heat_sum_density__1__50%__',
#         'arr_heat_sum_density__2__50%__',
#         'arr_heat_sum_density__3__50%__',
#         'arr_heat_sum_density__4__50%__',
        'arr_heat_sum_non_brain_vs_brain__50%__',
        'arr_heat_sum_0_vs_1_3_4__50%__',
        'arr_heat_sum_0_2_vs_1_3_4__50%__',
    ]:
        columns.append(f'{m}{cls}')
        
for cls in ['AD+CN']: 
# for cls in ['AD+CN', 'AD', 'CN']: 
    for m in [
#         'ins_del__auc_median__',
        'arr_heat_sum_density__0__50%__',
        'arr_heat_sum_density__1__50%__',
        'arr_heat_sum_density__2__50%__',
        'arr_heat_sum_density__3__50%__',
        'arr_heat_sum_density__4__50%__',
#         'arr_heat_sum_non_brain_vs_brain__50%__',
#         'arr_heat_sum_0_vs_1_3_4__50%__',
#         'arr_heat_sum_0_2_vs_1_3_4__50%__',
    ]:
        columns.append(f'{m}{cls}')
        
new_names = {}

for col in columns:
    new_name = col.replace('50%', '')
    new_name = new_name.replace('median', '')
    new_name = new_name.replace('AD', '(AD)')
    new_name = new_name.replace('CN', '(CN)')
    new_name = new_name.replace('(AD)+(CN)', '(AD + CN)')
    new_name = new_name.replace('_', ' ')
    new_name = new_name.replace('   ', ' ')
    new_name = new_name.replace('  ', ' ')
    new_name = new_name.replace('arr heat sum density 0', 'B - nie mozog')
    new_name = new_name.replace('arr heat sum density 1', 'B - šedá hmota')
    new_name = new_name.replace('arr heat sum density 2', 'B - biela hmota')
    new_name = new_name.replace('arr heat sum density 3', 'B - komory')
    new_name = new_name.replace('arr heat sum density 4', 'B - hipokampus')
    new_name = new_name.replace('ins del auc', 'A')
    new_name = new_name.replace('arr heat sum non brain vs brain', 'C')
    new_name = new_name.replace('arr heat sum 0 vs 1 3 4', 'E')
    new_name = new_name.replace('arr heat sum 0 2 vs 1 3 4', 'D')
    new_names[col] = new_name
        
def map_notebook_key(row):
    if 'captum--guided-backprop' in row['notebook_key']:
        row['notebook_key'] = 'Guided Backprop'
    if 'captum--guided-grad-cam' in row['notebook_key']:
        row['notebook_key'] = 'Guided GradCAM'
    if 'captum--grad-cam' in row['notebook_key']:
        row['notebook_key'] = 'GradCAM'
    if 'b2-value+1' in row['notebook_key']:
        row['notebook_key'] = 'RISE (b2_value = 1)'
    if 'b1+1' in row['notebook_key']:
        row['notebook_key'] = 'RISEI'
    if 'b2-value+0' in row['notebook_key']:
        row['notebook_key'] = 'RISE (b2_value = 0)'
    if 'guided' in row['notebook_key']:
        row['notebook_key'] = 'Guided RISE (b2_value = 0)'
    return row    
    
q_df = df[columns]

def fix_floats(row):
    for col in list(q_df.columns):
        print(col)
        if col != 'notebook_key':
            row[col] = '%0.3f' % row[col]
    return row

q_df = q_df.apply(fix_floats, axis=1)
q_df = q_df.apply(map_notebook_key, axis=1)
q_df = q_df.rename(columns=new_names)
new_columns = list(q_df.columns)
new_columns.sort()
new_columns.remove('notebook key')
new_columns.insert(0, 'notebook key')

q_df = q_df[new_columns]
q_df = q_df.sort_values(by='notebook key')
q_df = q_df.rename(columns={'notebook key': 'Metóda'})
q_df

notebook_key
ins_del__auc_median__AD+CN
arr_heat_sum_non_brain_vs_brain__50%__AD+CN
arr_heat_sum_0_vs_1_3_4__50%__AD+CN
arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN
ins_del__auc_median__AD
arr_heat_sum_non_brain_vs_brain__50%__AD
arr_heat_sum_0_vs_1_3_4__50%__AD
arr_heat_sum_0_2_vs_1_3_4__50%__AD
ins_del__auc_median__CN
arr_heat_sum_non_brain_vs_brain__50%__CN
arr_heat_sum_0_vs_1_3_4__50%__CN
arr_heat_sum_0_2_vs_1_3_4__50%__CN
arr_heat_sum_density__0__50%__AD+CN
arr_heat_sum_density__1__50%__AD+CN
arr_heat_sum_density__2__50%__AD+CN
arr_heat_sum_density__3__50%__AD+CN
arr_heat_sum_density__4__50%__AD+CN
notebook_key
ins_del__auc_median__AD+CN
arr_heat_sum_non_brain_vs_brain__50%__AD+CN
arr_heat_sum_0_vs_1_3_4__50%__AD+CN
arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN
ins_del__auc_median__AD
arr_heat_sum_non_brain_vs_brain__50%__AD
arr_heat_sum_0_vs_1_3_4__50%__AD
arr_heat_sum_0_2_vs_1_3_4__50%__AD
ins_del__auc_median__CN
arr_heat_sum_non_brain_vs_brain__50%__CN
arr_heat_sum_0_vs_1_3_4__50%__CN
arr_hea

,Metóda,A (AD + CN),A (AD),A (CN),B - biela hmota (AD + CN),B - hipokampus (AD + CN),B - komory (AD + CN),B - nie mozog (AD + CN),B - šedá hmota (AD + CN),C (AD + CN),C (AD),C (CN),D (AD + CN),D (AD),D (CN),E (AD + CN),E (AD),E (CN)
2,GradCAM,0.572,0.606,0.541,0.466,0.400,0.417,0.503,0.452,1.108,0.854,1.234,1.118,0.948,1.171,1.130,0.910,1.206
0,Guided Backprop,0.678,0.803,0.564,0.607,0.559,0.333,0.464,0.572,0.808,0.810,0.802,0.875,0.882,0.868,0.836,0.840,0.828
1,Guided GradCAM,0.673,0.810,0.538,0.507,0.500,0.491,0.498,0.504,0.986,0.942,0.999,0.991,0.970,0.998,0.989,0.957,0.998
6,Guided RISE (b2_value = 0),0.547,0.552,0.473,0.606,0.556,0.331,0.464,0.571,0.809,0.810,0.803,0.875,0.882,0.869,0.837,0.841,0.830
4,RISE (b2_value = 0),0.401,0.372,0.426,0.497,0.491,0.494,0.502,0.499,1.014,0.985,1.015,1.000,0.991,1.004,1.004,0.984,1.006
3,RISE (b2_value = 1),0.610,0.550,0.638,0.512,0.435,0.537,0.494,0.504,0.970,0.996,0.968,0.989,1.005,0.983,0.982,1.002,0.975
5,RISEI,0.443,0.423,0.462,0.498,0.501,0.484,0.500,0.503,1.002,1.004,0.999,0.999,0.990,1.005,0.995,0.993,1.002


In [216]:
q_df.to_csv('evaluation.csv', index=False)

In [189]:
new_names

{'notebook_key': 'notebook key',
 'ins_del__auc_median__AD+CN': 'A (AD + CN)',
 'arr_heat_sum_non_brain_vs_brain__50%__AD+CN': 'C (AD + CN)',
 'arr_heat_sum_0_vs_1_3_4__50%__AD+CN': 'E (AD + CN)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN': 'D (AD + CN)',
 'ins_del__auc_median__AD': 'A (AD)',
 'arr_heat_sum_non_brain_vs_brain__50%__AD': 'C (AD)',
 'arr_heat_sum_0_vs_1_3_4__50%__AD': 'E (AD)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__AD': 'D (AD)',
 'ins_del__auc_median__CN': 'A (CN)',
 'arr_heat_sum_non_brain_vs_brain__50%__CN': 'C (CN)',
 'arr_heat_sum_0_vs_1_3_4__50%__CN': 'E (CN)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__CN': 'D (CN)',
 'arr_heat_sum_density__0__50%__AD+CN': 'B - nie mozog (AD + CN)',
 'arr_heat_sum_density__1__50%__AD+CN': 'B - šedá hmota (AD + CN)',
 'arr_heat_sum_density__2__50%__AD+CN': 'B - biela hmota (AD + CN)',
 'arr_heat_sum_density__3__50%__AD+CN': 'B - komory (AD + CN)',
 'arr_heat_sum_density__4__50%__AD+CN': 'B - hipokampus (AD + CN)'}

In [37]:
q_df

,ins_del__auc_median__AD+CN,arr_heat_sum_density__1__50%__AD+CN,arr_heat_sum_non_brain_vs_brain__50%__AD+CN,arr_heat_sum_0_vs_1_3_4__50%__AD+CN,arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN,ins_del__auc_median__AD,arr_heat_sum_density__1__50%__AD,arr_heat_sum_non_brain_vs_brain__50%__AD,arr_heat_sum_0_vs_1_3_4__50%__AD,arr_heat_sum_0_2_vs_1_3_4__50%__AD,ins_del__auc_median__CN,arr_heat_sum_density__1__50%__CN,arr_heat_sum_non_brain_vs_brain__50%__CN,arr_heat_sum_0_vs_1_3_4__50%__CN,arr_heat_sum_0_2_vs_1_3_4__50%__CN
0,0.677949,0.029566,0.590628,0.642198,0.752738,0.677949,0.032723,0.590628,0.654175,0.762630,0.677949,0.026495,0.585857,0.636430,0.726872
1,0.673344,0.005113,1.190229,1.216895,1.195227,0.673344,0.012750,0.581095,0.703406,0.840003,0.673344,0.001033,4.919350,3.528462,3.031824
2,0.443367,0.511825,1.001478,0.995809,0.998478,0.443367,0.516813,1.002631,0.994592,0.994044,0.443367,0.502769,0.999248,0.998969,1.002076
3,0.571740,0.061006,1.599122,1.491213,1.372249,0.571740,0.189713,0.821046,0.892887,0.940256,0.571740,0.011814,6.593994,5.032570,4.257320
4,0.610127,0.471464,0.980864,0.987395,0.992803,0.610127,0.476230,1.000142,1.001504,1.003403,0.610127,0.466219,0.979799,0.983250,0.988741


In [11]:
new_data = {'notebook_key': [], 'metric': [], 'value': []}
df = saver.df.set_index('notebook_key')
for notebook_key, values in df.iterrows():
    for key, value in values.items():
        new_data['notebook_key'].append(notebook_key)
        new_data['metric'].append(key)
        new_data['value'].append(value)

new_df = pd.DataFrame(data=new_data)

In [12]:
new_df.head()

,notebook_key,metric,value
0,captum--guided-backprop,y_true__mean__AD+CN,0.500000
1,captum--guided-backprop,y_pred__mean__AD+CN,0.488614
2,captum--guided-backprop,arr_heat_sum__0__mean__AD+CN,17540.222557
3,captum--guided-backprop,arr_heat_sum__1__mean__AD+CN,7996.123205
4,captum--guided-backprop,arr_heat_sum__2__mean__AD+CN,6356.979385


In [159]:
# higher is better
new_df[new_df['metric'] == 'insertion__auc_median']

,notebook_key,metric,value
129,captum--guided-grad-cam,insertion__auc_median,0.819407
271,captum--grad-cam,insertion__auc_median,0.647560
413,risei--3d-cnn--m+2048--b1+0--b2+1--s+8--p1+0.25,insertion__auc_median,0.595248
555,risei--3d-cnn--m+2048--b1+0--b2+1--s+16--p1+0.25,insertion__auc_median,0.588939
697,captum--guided-backprop,insertion__auc_median,0.773093


In [160]:
# lower is better
new_df[new_df['metric'] == 'deletion__auc_median']

,notebook_key,metric,value
137,captum--guided-grad-cam,deletion__auc_median,0.524651
279,captum--grad-cam,deletion__auc_median,0.513192
421,risei--3d-cnn--m+2048--b1+0--b2+1--s+8--p1+0.25,deletion__auc_median,0.565032
563,risei--3d-cnn--m+2048--b1+0--b2+1--s+16--p1+0.25,deletion__auc_median,0.570917
705,captum--guided-backprop,deletion__auc_median,0.348439


In [161]:
# gain brain vs non-brain areas, higher is better
new_df[new_df['metric'] == 'arr_heat_sum_gain_other__50%']

,notebook_key,metric,value
89,captum--guided-grad-cam,arr_heat_sum_gain_other__50%,0.646925
231,captum--grad-cam,arr_heat_sum_gain_other__50%,0.689266
373,risei--3d-cnn--m+2048--b1+0--b2+1--s+8--p1+0.25,arr_heat_sum_gain_other__50%,0.997145
515,risei--3d-cnn--m+2048--b1+0--b2+1--s+16--p1+0.25,arr_heat_sum_gain_other__50%,0.998381
657,captum--guided-backprop,arr_heat_sum_gain_other__50%,1.048978
